<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=63f899e4685aefbf400eeb1eb57616b8b5aa2fb1b4bd579a8dfb752b2c024158
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 11:47:19
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -46.06 K (-0.32%)
Current PnL: -23.80 L (-15.6%)
CY Booked + Current PnL: -9.91 L (-6.5%)
-------------------
Total profit:  1.20 L
Total loss:  -25.01 L
-------------------
Total Booked + Current PnL: 17.07 L (13.58%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.78%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.21 L (62.83%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.04,-14.19,16.64,0.09,14390.0,-14297.0,86480.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.62,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.38,-9.67,10.73,0.02,16918.0,-16887.0,157672.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.15,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.02,6.59,12.76,20.19,23709.0,11485.0,185805.0,-16.49,50.0,X-MC,7.57,68.0,0.48,1.31,22.70,X40,ATH,INSURANCE
43,ITC,452.00,-0.19,-1.58,12.21,10.44,24050.0,-3169.0,196969.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.39,3.89,X40,NTT,FMCG
83,VOLTAS,1530.00,0.09,6.06,12.85,19.69,26133.0,11628.0,203370.0,-4.26,46.0,X-MC,3.44,78.0,0.44,1.43,13.27,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,4.04,-49.30,233.44,69.05,157486.0,-65603.0,67463.0,-58.98,38.0,H-SC,23.55,140.0,-0.42,0.48,4.04,SR,ATH,IT
58,RAJOOENG,143.10,2.52,-29.03,96.73,39.61,70361.0,-29760.0,72740.0,-52.00,26.0,H-SC,20.02,135.0,-0.42,0.51,2.52,AR,ATH,MISC
18,COFFEEDAY,80.00,2.15,-38.16,118.76,35.27,83386.0,-43335.0,70214.0,-51.71,36.0,L-SC,17.79,270.0,-0.52,0.49,71.85,XR,NTT,HOTELS
61,REPCOHOME,880.00,2.14,-11.13,113.59,89.82,289218.0,-31889.0,254616.0,-55.29,49.0,H-SC,10.40,160.0,-0.11,1.79,34.71,XY24,NTT,FINANCE
6,ASIANTILES,137.00,1.71,-11.75,105.86,81.67,87639.0,-11022.0,82788.0,7444.44,55.0,L-SC,18.83,271.0,-0.13,0.58,60.32,XR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-5.21,-16.72,84.31,53.50,211878.0,-50443.0,251308.0,-59.67,41.0,X-MC,4.06,58.0,-0.24,1.77,11.62,XY24,NTT,FMCG
34,HONAUT,58357.33,-3.52,-14.07,66.78,43.32,93465.0,-22912.0,139960.0,-26.21,43.0,X-SC,2.01,90.0,-0.25,0.99,7.46,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-2.49,-21.58,109.32,64.15,203991.0,-51351.0,186600.0,-54.13,53.0,X-SC,1.64,82.0,-0.25,1.31,4.67,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-2.21,-63.84,176.71,0.07,87662.0,-87569.0,49608.0,1718.18,53.0,L-SC,1.53,268.0,-1.00,0.35,23.38,OX40N,NTT,JEWELLERY
50,LTIM,7201.88,-1.77,6.34,23.81,31.66,62326.0,15614.0,261765.0,2.25,55.0,H-LC,12.69,43.0,0.25,1.84,45.24,X200,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.42,-0.42,107.6,106.72,132151.0,-519.0,122817.0,-50.63,34.0,H-SC,3.71,139.0,-0.0,0.87,22.86,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.11,-5.17,119.25,107.91,166958.0,-7637.0,140007.0,-22.98,33.0,M-SC,10.48,216.0,-0.05,0.99,0.11,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.38,-9.67,10.73,0.02,16918.0,-16887.0,157672.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.15,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.04,-14.19,16.64,0.09,14390.0,-14297.0,86480.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.62,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.76,-23.17,47.68,13.47,98792.0,-62469.0,207198.0,-68.51,25.0,H-SC,1.38,158.0,-0.63,1.46,5.35,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.35,-23.47,30.66,-0.01,29987.0,-30001.0,97804.0,19921.74,55.0,M-SC,1.67,233.0,-1.00,0.69,10.75,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.24,1.17,70.27,72.27,120468.0,1988.0,171436.0,-13.34,55.0,M-LC,4.53,99.0,0.02,1.21,7.42,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.24,1.17,70.27,72.27,120468.0,1988.0,171436.0,-13.34,55.0,M-LC,4.53,99.0,0.02,1.21,7.42,XR,NTT,IT
38,INDIAMART,4810.62,0.42,-0.42,107.60,106.72,132151.0,-519.0,122817.0,-50.63,34.0,H-SC,3.71,139.0,-0.00,0.87,22.86,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.55,-1.75,38.05,35.63,78347.0,-3668.0,205906.0,-15.41,37.0,H-MC,4.39,119.0,-0.05,1.45,14.63,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.11,-5.17,119.25,107.91,166958.0,-7637.0,140007.0,-22.98,33.0,M-SC,10.48,216.0,-0.05,0.99,0.11,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,0.29,-4.91,91.32,81.92,99683.0,-5640.0,109158.0,-44.29,33.0,M-SC,4.54,236.0,-0.06,0.77,29.23,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.45,-15.47,68.21,42.18,159364.0,-42772.0,233638.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.62,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,12.89,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.13,-9.25,26.97,15.23,49034.0,-18536.0,181811.0,-22.32,25.0,X-MC,6.64,66.0,-0.38,1.28,20.02,X40N,NTT,REALTY
60,RELAXO,1176.00,-0.22,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,27.0,X-SC,3.96,91.0,-0.48,0.53,1.16,X40N,NTT,FOOTWEAR
22,DIXON,18931.72,-1.06,-6.79,30.45,21.58,39773.0,-9522.0,130617.0,-53.04,29.0,X-MC,6.87,56.0,-0.24,0.92,15.64,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.45,-15.47,68.21,42.18,159364.0,-42772.0,233638.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.19,-1.58,12.21,10.44,24050.0,-3169.0,196969.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.39,3.89,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.51,1.20,29.41,30.97,52603.0,2124.0,178862.0,-22.64,49.0,X-MC,1.57,75.0,0.04,1.26,13.91,X40,ATH,INSURANCE
3,ACC,3906.0,-2.49,-21.58,109.32,64.15,203991.0,-51351.0,186600.0,-54.13,53.0,X-SC,1.64,82.0,-0.25,1.31,4.67,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.11,-14.90,47.49,25.51,75208.0,-27730.0,158365.0,-2.87,53.0,H-LC,1.87,49.0,-0.37,1.12,16.70,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.62,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,12.89,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.50,-13.93,72.48,48.45,125084.0,-27929.0,172577.0,-20.09,29.0,X-MC,6.83,64.0,-0.22,1.22,0.00,X40N,ATH,FINANCE
76,TMPV,600.00,-0.45,-15.47,68.21,42.18,159364.0,-42772.0,233638.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.65,0.00,XY24,NTT,AUTO
54,PAGEIND,51605.08,-0.71,-5.74,33.74,26.07,52074.0,-9400.0,154340.0,-30.55,33.0,X-MC,9.05,55.0,-0.18,1.09,0.05,X40,ATH,APPARELS
55,PGHH,17907.65,-0.47,-4.57,40.17,33.76,76982.0,-9180.0,191640.0,-32.64,33.0,X-MC,3.37,57.0,-0.12,1.35,0.39,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,1.44,-28.11,98.69,42.84,46123.0,-18271.0,46735.0,-53.17,40.0,X-SC,37.38,83.0,-0.40,0.33,1.69,XY24,NTT,MISC
60,RELAXO,1176.00,-0.22,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,27.0,X-SC,3.96,91.0,-0.48,0.53,1.16,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.62,-35.53,109.70,35.20,91005.0,-45712.0,82958.0,8.74,24.0,X-SC,12.89,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.06,-6.79,30.45,21.58,39773.0,-9522.0,130617.0,-53.04,29.0,X-MC,6.87,56.0,-0.24,0.92,15.64,X40N,ATH,IT
52,MEDANTA,1486.00,-1.16,3.20,20.23,24.08,26505.0,4066.0,131016.0,-5.17,46.0,X-SC,4.95,89.0,0.15,0.92,23.45,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.37,-14.97,40.28,19.28,118493.0,-51783.0,294173.0,-25.89,59.0,X-LC,7.53,1.0,-0.44,2.07,8.75,X40,ATH,IT
41,INFY,2275.00,-1.16,4.16,48.69,54.87,158676.0,13002.0,325890.0,-18.88,56.0,X-LC,3.01,2.0,0.08,2.30,12.78,X40,BTT,IT
76,TMPV,600.00,-0.45,-15.47,68.21,42.18,159364.0,-42772.0,233638.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.22,-9.69,50.22,35.67,143253.0,-30592.0,285250.0,-20.46,41.0,X-LC,5.95,4.0,-0.21,2.01,2.83,X40N,ATH,FMCG
43,ITC,452.00,-0.19,-1.58,12.21,10.44,24050.0,-3169.0,196969.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.39,3.89,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.71,-11.75,105.86,81.67,87639.0,-11022.0,82788.0,7444.44,55.0,L-SC,18.83,271.0,-0.13,0.58,60.32,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.90,-13.29,60.92,39.54,53438.0,-13440.0,87719.0,-32.23,63.0,M-SC,6.70,220.0,-0.25,0.62,23.69,AR,ATH,AUTO
51,MASFIN,398.61,0.02,-5.19,28.73,22.05,26689.0,-5085.0,92895.0,-18.25,50.0,H-SC,7.34,164.0,-0.19,0.65,35.03,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.34,-20.10,53.99,23.03,77246.0,-35996.0,143075.0,55.70,47.0,H-SC,10.58,167.0,-0.47,1.01,41.96,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.38,-9.67,10.73,0.02,16918.0,-16887.0,157672.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.15,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.38,-9.67,10.73,0.02,16918.0,-16887.0,157672.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.15,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.39,1.03,19.72,20.95,50159.0,2587.0,254355.0,0.07,77.0,X-LC,13.40,31.0,0.05,1.79,37.03,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.47,-38.57,116.32,32.89,55171.0,-29776.0,47430.0,-685.45,57.0,L-MC,5.55,259.0,-0.54,0.33,30.62,XR,NTT,BANKS
12,BERGEPAINT,680.00,-0.31,2.88,17.75,21.14,41515.0,6549.0,233888.0,-4.44,63.0,X-MC,3.90,74.0,0.16,1.65,31.83,XY24,NTT,PAINTS
0,5PAISA,593.00,-0.63,-36.57,78.00,12.90,108100.0,-79906.0,138590.0,104.33,55.0,H-SC,14.35,161.0,-0.74,0.98,15.24,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.70
1,25,44.59
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.67
MC    29.24
LC    25.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.97
X40      21.21
X40N     11.87
XR        9.63
XY25      9.06
AR        8.99
OX40N     7.68
X200      1.84
SR        1.00
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.96
X-MC    22.81
X-LC    19.98
M-SC    12.07
X-SC     8.22
H-MC     4.73
H-LC     2.96
L-SC     1.42
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.72,-5.99,40.27
IT,13.74,-16.00,75.89
FINANCE,9.46,-15.43,64.20
MISC,7.43,-23.42,73.98
PAINTS,6.01,-8.23,24.31
ELECTRICAL,5.93,-11.65,52.55
INSURANCE,4.45,-0.10,35.38
PHARMA,3.88,-1.48,33.96
AUTO,3.35,-21.01,72.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3067327.0,21
AR,1296798.0,10
XR,1293148.0,13
X40,1028353.0,14
X40N,763756.0,9
OX40N,703051.0,10
XY25,353680.0,6
SR,279144.0,2
MH,73879.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3579709.0,25
M-SC,1346266.0,15
X-MC,1246600.0,16
X-LC,926531.0,11
X-SC,751675.0,8
H-MC,407047.0,3
L-SC,258687.0,3
H-LC,137534.0,2
M-LC,120468.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1220618.0      6
           AR         880775.0      5
           XR         797079.0      7
M-SC       XY24       785002.0      6
X-MC       X40        449266.0      7
X-LC       X40        402401.0      5
X-MC       XY24       361803.0      3
H-SC       OX40N      328214.0      4
X-SC       X40N       298370.0      3
M-SC       OX40N      287175.0      5
H-SC       SR         279144.0      2
X-SC       XY24       276619.0      3
X-MC       X40N       251964.0      4
X-LC       XY24       232369.0      2
H-MC       AR         216131.0      2
X-LC       X40N       213422.0      2
H-MC       XY24       190916.0      1
X-MC       XY25       183567.0      2
X-SC       X40        176686.0      2
L-SC       XR         171025.0      2
M-SC       XR         149405.0      2
           AR         124684.0      2
M-LC       XR         120468.0      1
L-SC       OX40N       87662.0      1
X-LC       XY25        78339.0      2
H-LC       AR          75208.0      1
H-SC       MH          73879.0      1
H-LC       X200        62326.0      1
L-MC       XR          55171.0      1
M-MC       XY25        50355.0      1
L-LC       XY25        41419.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
